# Machine Learning Pipeline with tf.estimator

<img src="img/tf_estimator_diagram.png">

In [40]:
import time
import os
import numpy as np
import tensorflow as tf

# Premade Estimator
## 1. Import Data

In [41]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


In [42]:
train_data.shape

(55000, 784)

## 2. Create Input Functions

In [43]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},y=train_labels,batch_size=100,num_epochs=None,shuffle=True)
eval_input_fn  = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},y=eval_labels,num_epochs=1,shuffle=False)
pred_input_fn  = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},num_epochs=1,shuffle=False)

## 3. Create Estimator

###### Set paramters and model path

In [45]:
config     = tf.estimator.RunConfig(save_checkpoints_secs = 30, keep_checkpoint_max = 5)

#mnist_path = 'gs://tsaikevin-working/models/mnist/'
mnist_path = './mnist/'
dnn_dir = mnist_path + 'dnn_model/dnn_model'
dnn_dir = dnn_dir + '_' + time.strftime("%Y%m%d%H%M%S")
dnn_dir

'./mnist/dnn_model/dnn_model_20180423054240'

In [47]:
# DNNClassifier requires feature in a list
features = [tf.feature_column.numeric_column("x",shape=([784]),dtype=tf.float32)]

dnn_classifier = tf.estimator.DNNClassifier(hidden_units=[64,64],feature_columns=features,
                                            model_dir=dnn_dir,n_classes=10,config=config)

INFO:tensorflow:Using config: {'_evaluation_master': '', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f380c9c7160>, '_session_config': None, '_save_checkpoints_secs': 30, '_tf_random_seed': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_log_step_count_steps': 100, '_num_ps_replicas': 0, '_task_id': 0, '_master': '', '_is_chief': True, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': './mnist/dnn_model/dnn_model_20180423054240', '_global_id_in_cluster': 0, '_num_worker_replicas': 1, '_service': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None}


## 4. Create TrainSpec and EvalSpec

In [50]:
train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, max_steps=10000)
eval_spec  = tf.estimator.EvalSpec(input_fn=eval_input_fn)

## 5. Run Train and Evaluate

In [51]:
!rm -rf $dnn_dir
tf.estimator.train_and_evaluate(dnn_classifier, train_spec, eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 600 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./mnist/dnn_model/dnn_model_20180423054240/model.ckpt.
INFO:tensorflow:step = 0, loss = 239.03842
INFO:tensorflow:global_step/sec: 451.765
INFO:tensorflow:step = 100, loss = 38.427498 (0.222 sec)
INFO:tensorflow:global_step/sec: 598.303
INFO:tensorflow:step = 200, loss = 30.084812 (0.167 sec)
INFO:tensorflow:global_step/sec: 599.886
INFO:tensorflow:step = 300, loss = 23.06752 (0.167 sec)
INFO:tensorflow:global_step/sec: 594.678
INFO:tensorflow:step = 400, loss = 16.682331 (0.168 sec)
INFO:tensorflow

INFO:tensorflow:step = 7800, loss = 2.631443 (0.165 sec)
INFO:tensorflow:global_step/sec: 599.614
INFO:tensorflow:step = 7900, loss = 1.7197607 (0.167 sec)
INFO:tensorflow:global_step/sec: 602.953
INFO:tensorflow:step = 8000, loss = 1.4172525 (0.166 sec)
INFO:tensorflow:global_step/sec: 611.181
INFO:tensorflow:step = 8100, loss = 6.3458805 (0.164 sec)
INFO:tensorflow:global_step/sec: 606.997
INFO:tensorflow:step = 8200, loss = 11.199628 (0.165 sec)
INFO:tensorflow:global_step/sec: 607.398
INFO:tensorflow:step = 8300, loss = 1.3805354 (0.165 sec)
INFO:tensorflow:global_step/sec: 594.704
INFO:tensorflow:step = 8400, loss = 4.6328006 (0.168 sec)
INFO:tensorflow:global_step/sec: 607.578
INFO:tensorflow:step = 8500, loss = 1.6875553 (0.165 sec)
INFO:tensorflow:global_step/sec: 603.35
INFO:tensorflow:step = 8600, loss = 4.194741 (0.166 sec)
INFO:tensorflow:global_step/sec: 606.102
INFO:tensorflow:step = 8700, loss = 1.0767858 (0.165 sec)
INFO:tensorflow:global_step/sec: 602.738
INFO:tensorfl

###### Contents of model_dir

In [54]:
!ls -l $dnn_dir

total 2176
-rw-rw-r-- 1 tsaikevin tsaikevin    132 Apr 23 05:44 checkpoint
drwxr-xr-x 2 tsaikevin tsaikevin   4096 Apr 23 05:44 eval
-rw-rw-r-- 1 tsaikevin tsaikevin 883076 Apr 23 05:44 events.out.tfevents.1524462235.tfgpu2
-rw-rw-r-- 1 tsaikevin tsaikevin 234330 Apr 23 05:43 graph.pbtxt
-rw-rw-r-- 1 tsaikevin tsaikevin 440408 Apr 23 05:44 model.ckpt-10000.data-00000-of-00001
-rw-rw-r-- 1 tsaikevin tsaikevin    559 Apr 23 05:44 model.ckpt-10000.index
-rw-rw-r-- 1 tsaikevin tsaikevin 102023 Apr 23 05:44 model.ckpt-10000.meta
-rw-rw-r-- 1 tsaikevin tsaikevin 440408 Apr 23 05:43 model.ckpt-1.data-00000-of-00001
-rw-rw-r-- 1 tsaikevin tsaikevin    559 Apr 23 05:43 model.ckpt-1.index
-rw-rw-r-- 1 tsaikevin tsaikevin 102023 Apr 23 05:43 model.ckpt-1.meta


###### Rerun same train_and_evaluate command

In [55]:
tf.estimator.train_and_evaluate(dnn_classifier, train_spec, eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 600 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Skipping training since max_steps has already saved.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-23-05:46:53
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./mnist/dnn_model/dnn_model_20180423054240/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [60/100]
INFO:tensorflow:Evaluation [70/100]
INFO:tensorflow:Finished evaluation at 2018-04-23-05:46:53
INFO:tensorflow:Saving dict for global step 10000: ac

###### Rerun command with higher max_steps

In [56]:
train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, max_steps=20000)
tf.estimator.train_and_evaluate(dnn_classifier, train_spec, eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 600 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./mnist/dnn_model/dnn_model_20180423054240/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 10001 into ./mnist/dnn_model/dnn_model_20180423054240/model.ckpt.
INFO:tensorflow:step = 10000, loss = 3.733982
INFO:tensorflow:global_step/sec: 465.531
INFO:tensorflow:step = 10100, loss = 2.7542496 (0.216 sec)
INFO:tensorflow:global_step/sec: 589.602
INFO:tensorflow:step = 10200, loss = 17.294672 (0.169 sec)
INFO:tensorflow:global_step/sec: 594.814
INFO:tensorflow:step = 10300, loss = 3.1166909 (0.168 se

INFO:tensorflow:step = 17600, loss = 0.49716434 (0.167 sec)
INFO:tensorflow:global_step/sec: 600.917
INFO:tensorflow:step = 17700, loss = 0.82697284 (0.166 sec)
INFO:tensorflow:global_step/sec: 595.839
INFO:tensorflow:step = 17800, loss = 1.373048 (0.168 sec)
INFO:tensorflow:global_step/sec: 592.096
INFO:tensorflow:step = 17900, loss = 0.6136706 (0.169 sec)
INFO:tensorflow:global_step/sec: 605.507
INFO:tensorflow:step = 18000, loss = 0.8859979 (0.165 sec)
INFO:tensorflow:global_step/sec: 607.193
INFO:tensorflow:step = 18100, loss = 0.6409297 (0.164 sec)
INFO:tensorflow:global_step/sec: 599.609
INFO:tensorflow:step = 18200, loss = 3.2097597 (0.167 sec)
INFO:tensorflow:global_step/sec: 600.825
INFO:tensorflow:step = 18300, loss = 0.8463269 (0.166 sec)
INFO:tensorflow:global_step/sec: 598.279
INFO:tensorflow:step = 18400, loss = 0.9659989 (0.167 sec)
INFO:tensorflow:global_step/sec: 589.54
INFO:tensorflow:step = 18500, loss = 3.69181 (0.169 sec)
INFO:tensorflow:global_step/sec: 603.692
IN

In [57]:
!ls -l $dnn_dir
!cat $dnn_dir/checkpoint

total 3980
-rw-rw-r-- 1 tsaikevin tsaikevin     226 Apr 23 05:47 checkpoint
drwxr-xr-x 2 tsaikevin tsaikevin    4096 Apr 23 05:44 eval
-rw-rw-r-- 1 tsaikevin tsaikevin 1632234 Apr 23 05:47 events.out.tfevents.1524462235.tfgpu2
-rw-rw-r-- 1 tsaikevin tsaikevin  234330 Apr 23 05:47 graph.pbtxt
-rw-rw-r-- 1 tsaikevin tsaikevin  440408 Apr 23 05:44 model.ckpt-10000.data-00000-of-00001
-rw-rw-r-- 1 tsaikevin tsaikevin     559 Apr 23 05:44 model.ckpt-10000.index
-rw-rw-r-- 1 tsaikevin tsaikevin  102023 Apr 23 05:44 model.ckpt-10000.meta
-rw-rw-r-- 1 tsaikevin tsaikevin  440408 Apr 23 05:47 model.ckpt-10001.data-00000-of-00001
-rw-rw-r-- 1 tsaikevin tsaikevin     559 Apr 23 05:47 model.ckpt-10001.index
-rw-rw-r-- 1 tsaikevin tsaikevin  102023 Apr 23 05:47 model.ckpt-10001.meta
-rw-rw-r-- 1 tsaikevin tsaikevin  440408 Apr 23 05:43 model.ckpt-1.data-00000-of-00001
-rw-rw-r-- 1 tsaikevin tsaikevin     559 Apr 23 05:43 model.ckpt-1.index
-rw-rw-r-- 1 tsaikevin tsaikevin  102023 Apr 23 05:43 model

## 6. Run Prediction

In [58]:
predictions = list(dnn_classifier.predict(input_fn=pred_input_fn))
predictions[0]

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./mnist/dnn_model/dnn_model_20180423054240/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


{'class_ids': array([7]),
 'classes': array([b'7'], dtype=object),
 'logits': array([ -8.096183 ,   2.485023 ,   2.9221532,   2.9454515,  -9.212211 ,
         -2.2594404, -24.286165 ,  17.192411 ,  -2.6585417,  -1.4273846],
       dtype=float32),
 'probabilities': array([1.04064535e-11, 4.09884848e-07, 6.34607204e-07, 6.49566005e-07,
        3.40892939e-12, 3.56588847e-09, 9.68461303e-19, 9.99998331e-01,
        2.39243558e-09, 8.19453483e-09], dtype=float32)}

## 7. Export Model

In [59]:
def serving_input_receiver_fn():
    receiver_tensor = {'x': tf.placeholder(shape=[None,28,28,1], dtype=tf.string)}
    features = {'x': receiver_tensor['x']}
    return tf.estimator.export.ServingInputReceiver(features, receiver_tensor)

dnn_classifier.export_savedmodel(
    export_dir_base=dnn_dir,serving_input_receiver_fn=serving_input_receiver_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Classify: ['serving_default', 'classification']
INFO:tensorflow:Restoring parameters from ./mnist/dnn_model/dnn_model_20180423054240/model.ckpt-20000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b"./mnist/dnn_model/dnn_model_20180423054240/temp-b'1524462529'/saved_model.pb"


b'./mnist/dnn_model/dnn_model_20180423054240/1524462529'

# Using Custom Estimator
## 3. Create Estimator

In [60]:
# Convolution Block

def _conv(x,kernel,name,log=False):
    with tf.name_scope(name):
        W = tf.Variable(tf.truncated_normal(shape=kernel,stddev=0.01),name='W')
        b = tf.Variable(tf.constant(0.0,shape=[kernel[3]]),name='b')
        conv = tf.nn.conv2d(x, W, strides=[1,1,1,1],padding='SAME')
        activation = tf.nn.relu(tf.add(conv,b))
        pool = tf.nn.max_pool(activation,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
        if log==True:
            tf.summary.histogram("weights",W)
            tf.summary.histogram("biases",b)
            tf.summary.histogram("activations",activation)
        return pool

# Dense Block

def _dense(x,size_in,size_out,name,relu=False,log=False):
    with tf.name_scope(name):
        flat = tf.reshape(x,[-1,size_in])
        W = tf.Variable(tf.truncated_normal([size_in,size_out],stddev=0.1),name='W')
        b = tf.Variable(tf.constant(0.0,shape=[size_out]),name='b')
        activation = tf.add(tf.matmul(flat,W),b)
        if relu==True:
            activation = tf.nn.relu(activation)
        if log==True:
            tf.summary.histogram("weights",W)
            tf.summary.histogram("biases",b)
            tf.summary.histogram("activations",activation)
        return activation

In [61]:
def cnn_model_fn(features, labels, mode, params):
    
    #### 1 INFERNCE MODEL
    
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])
    conv1 = _conv(input_layer,kernel=[5,5,1,64],name='conv1',log=params['log'])
    conv2 = _conv(conv1,kernel=[5,5,64,64],name='conv2',log=params['log'])
    dense = _dense(conv2,size_in=7*7*64,size_out=params['dense_units'],
                   name='Dense',relu=True,log=params['log'])
    if mode==tf.estimator.ModeKeys.TRAIN:
        dense = tf.nn.dropout(dense,params['drop_out'])
    logits = _dense(dense,size_in=params['dense_units'],
                    size_out=10,name='Output',relu=False,log=params['log'])
    
    #### 2 CALCULATIONS AND METRICS
    
    predictions = {"classes": tf.argmax(input=logits,axis=1),
                   "logits": logits,
                   "probabilities": tf.nn.softmax(logits,name='softmax')}
    export_outputs = {'predictions': tf.estimator.export.PredictOutput(predictions)}
    if (mode==tf.estimator.ModeKeys.TRAIN or mode==tf.estimator.ModeKeys.EVAL):
        loss = tf.losses.sparse_softmax_cross_entropy(labels=labels,logits=logits)
        accuracy = tf.metrics.accuracy(
            labels=labels, predictions=tf.argmax(logits,axis=1))
        metrics = {'accuracy':accuracy}
        
    #### 3 MODE = PREDICT
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(
            mode=mode, predictions=predictions, export_outputs=export_outputs)

    #### 4 MODE = TRAIN

    if mode == tf.estimator.ModeKeys.TRAIN:
        learning_rate = tf.train.exponential_decay(
            params['learning_rate'],tf.train.get_global_step(),
            decay_steps=100000,decay_rate=0.96)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
        if params['replicate']==True:
            optimizer = tf.contrib.estimator.TowerOptimizer(optimizer)
        train_op = optimizer.minimize(loss=loss,global_step=tf.train.get_global_step())
        tf.summary.scalar('learning_rate', learning_rate)
        tf.summary.scalar('accuracy',accuracy[1])
        return tf.estimator.EstimatorSpec(
            mode=mode, loss=loss, train_op=train_op)
    
    #### 5 MODE = EVAL
    
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(
            mode=mode,loss=loss,eval_metric_ops=metrics)

###### Set paramters and model path¶

In [62]:
model_params  = {'drop_out'      : 0.8,
                 'dense_units'   : 1024,
                 'learning_rate' : 1e-3,
                 'log'           : True,
                 'replicate'     : False
                }

name = 'cnn_model_'

In [63]:
if model_params['replicate']==True:
    cnn_model_fn = tf.contrib.estimator.replicate_model_fn(
        cnn_model_fn, loss_reduction=tf.losses.Reduction.MEAN)

name = 'cnn_model/cnn_model_'
if model_params['replicate']==True:
    name = 'cnn_model_dist/cnn_model_'
name = name + 'dense(' + str(model_params['dense_units']) + ')_'
name = name + 'drop(' + str(model_params['drop_out']) + ')_'
name = name + 'lr(' + str(model_params['learning_rate']) + ')_'
name = name + time.strftime("%Y%m%d%H%M%S")
cnn_dir  = os.path.join(mnist_path,name)

print(cnn_dir)

cnn_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn,model_dir=cnn_dir,params=model_params,config=config)

./mnist/cnn_model/cnn_model_dense(1024)_drop(0.8)_lr(0.001)_20180423055032
INFO:tensorflow:Using config: {'_evaluation_master': '', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3807739b38>, '_session_config': None, '_save_checkpoints_secs': 30, '_tf_random_seed': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_log_step_count_steps': 100, '_num_ps_replicas': 0, '_task_id': 0, '_master': '', '_is_chief': True, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': './mnist/cnn_model/cnn_model_dense(1024)_drop(0.8)_lr(0.001)_20180423055032', '_global_id_in_cluster': 0, '_num_worker_replicas': 1, '_service': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None}


## 4. Create TrainSpec and EvalSpec

In [64]:
train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, max_steps=20000)
eval_spec  = tf.estimator.EvalSpec(input_fn=eval_input_fn,throttle_secs=10)

## 5. Run Train and Evaluate

In [65]:
tf.estimator.train_and_evaluate(cnn_classifier, train_spec, eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 10 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./mnist/cnn_model/cnn_model_dense(1024)_drop(0.8)_lr(0.001)_20180423055032/model.ckpt.
INFO:tensorflow:step = 0, loss = 2.3135552
INFO:tensorflow:global_step/sec: 135.991
INFO:tensorflow:step = 100, loss = 2.2459855 (0.737 sec)
INFO:tensorflow:global_step/sec: 149.796
INFO:tensorflow:step = 200, loss = 2.1701381 (0.668 sec)
INFO:tensorflow:global_step/sec: 149.815
INFO:tensorflow:step = 300, loss = 2.02536 (0.667 sec)
INFO:tensorflow:global_step/sec: 152.912
INFO:tensorflow:step = 400, loss = 1.70672

INFO:tensorflow:Restoring parameters from ./mnist/cnn_model/cnn_model_dense(1024)_drop(0.8)_lr(0.001)_20180423055032/model.ckpt-4227
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [60/100]
INFO:tensorflow:Evaluation [70/100]
INFO:tensorflow:Finished evaluation at 2018-04-23-05:51:12
INFO:tensorflow:Saving dict for global step 4227: accuracy = 0.9461, global_step = 4227, loss = 0.19281727
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./mnist/cnn_model/cnn_model_dense(1024)_drop(0.8)_lr(0.001)_20180423055032/model.ckpt-4227
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_

INFO:tensorflow:global_step/sec: 151.752
INFO:tensorflow:step = 8340, loss = 0.07737245 (0.659 sec)
INFO:tensorflow:global_step/sec: 151.48
INFO:tensorflow:step = 8440, loss = 0.1440635 (0.661 sec)
INFO:tensorflow:Saving checkpoints for 8450 into ./mnist/cnn_model/cnn_model_dense(1024)_drop(0.8)_lr(0.001)_20180423055032/model.ckpt.
INFO:tensorflow:Loss for final step: 0.14540976.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-23-05:51:44
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./mnist/cnn_model/cnn_model_dense(1024)_drop(0.8)_lr(0.001)_20180423055032/model.ckpt-8450
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [60/100]
INFO:tensorflow:

INFO:tensorflow:global_step/sec: 151.43
INFO:tensorflow:step = 11961, loss = 0.38789266 (0.660 sec)
INFO:tensorflow:global_step/sec: 149.648
INFO:tensorflow:step = 12061, loss = 0.12766789 (0.668 sec)
INFO:tensorflow:global_step/sec: 153.883
INFO:tensorflow:step = 12161, loss = 0.16826957 (0.650 sec)
INFO:tensorflow:global_step/sec: 152.302
INFO:tensorflow:step = 12261, loss = 0.060084004 (0.657 sec)
INFO:tensorflow:global_step/sec: 149.398
INFO:tensorflow:step = 12361, loss = 0.11866563 (0.669 sec)
INFO:tensorflow:global_step/sec: 150.429
INFO:tensorflow:step = 12461, loss = 0.16115314 (0.665 sec)
INFO:tensorflow:global_step/sec: 151.256
INFO:tensorflow:step = 12561, loss = 0.14937836 (0.661 sec)
INFO:tensorflow:global_step/sec: 151.62
INFO:tensorflow:step = 12661, loss = 0.13035113 (0.660 sec)
INFO:tensorflow:Saving checkpoints for 12673 into ./mnist/cnn_model/cnn_model_dense(1024)_drop(0.8)_lr(0.001)_20180423055032/model.ckpt.
INFO:tensorflow:Loss for final step: 0.10965171.
INFO:te

INFO:tensorflow:step = 15481, loss = 0.09919063
INFO:tensorflow:global_step/sec: 139.326
INFO:tensorflow:step = 15581, loss = 0.11945122 (0.719 sec)
INFO:tensorflow:global_step/sec: 148.438
INFO:tensorflow:step = 15681, loss = 0.11202935 (0.674 sec)
INFO:tensorflow:global_step/sec: 150.078
INFO:tensorflow:step = 15781, loss = 0.07611239 (0.666 sec)
INFO:tensorflow:global_step/sec: 151.706
INFO:tensorflow:step = 15881, loss = 0.06366579 (0.659 sec)
INFO:tensorflow:global_step/sec: 151.562
INFO:tensorflow:step = 15981, loss = 0.11152657 (0.660 sec)
INFO:tensorflow:global_step/sec: 150.064
INFO:tensorflow:step = 16081, loss = 0.0787029 (0.666 sec)
INFO:tensorflow:global_step/sec: 150.847
INFO:tensorflow:step = 16181, loss = 0.06699618 (0.663 sec)
INFO:tensorflow:global_step/sec: 152.01
INFO:tensorflow:step = 16281, loss = 0.08964717 (0.657 sec)
INFO:tensorflow:global_step/sec: 150.795
INFO:tensorflow:step = 16381, loss = 0.1315429 (0.663 sec)
INFO:tensorflow:global_step/sec: 154.249
INFO:

INFO:tensorflow:Finished evaluation at 2018-04-23-05:53:12
INFO:tensorflow:Saving dict for global step 19697: accuracy = 0.9757, global_step = 19697, loss = 0.07982718
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./mnist/cnn_model/cnn_model_dense(1024)_drop(0.8)_lr(0.001)_20180423055032/model.ckpt-19697
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 19698 into ./mnist/cnn_model/cnn_model_dense(1024)_drop(0.8)_lr(0.001)_20180423055032/model.ckpt.
INFO:tensorflow:step = 19697, loss = 0.16083923
INFO:tensorflow:global_step/sec: 139.177
INFO:tensorflow:step = 19797, loss = 0.18879715 (0.720 sec)
INFO:tensorflow:global_step/sec: 148.924
INFO:tensorflow:step = 19897, loss = 0.17482048 (0.671 sec)
INFO:tensorflow:global_step/sec: 150.285
INFO:tensorflow:step = 19997, los

## 6. Run Prediction

In [29]:
predictions = list(cnn_classifier.predict(input_fn=pred_input_fn))
predictions[0]

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./mnist/cnn_model_dist/cnn_model_dense(1024)_drop(0.3)_lr(0.001)_20180423022431/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


{'classes': 7,
 'logits': array([ 0.13511553, -1.6150415 , -0.84223723,  0.94165844, -1.844843  ,
        -0.57826984, -4.946539  , 10.389465  ,  0.13603215,  3.1536274 ],
       dtype=float32),
 'probabilities': array([3.5171990e-05, 6.1110140e-06, 1.3235424e-05, 7.8790501e-05,
        4.8563711e-06, 1.7233660e-05, 2.1840489e-07, 9.9908960e-01,
        3.5204241e-05, 7.1964774e-04], dtype=float32)}

## 7. Export Model

In [ ]:
def serving_input_receiver_fn():
    receiver_tensor = {'x': tf.placeholder(shape=[None,28,28,1], dtype=tf.float32)}
    features = {'x': receiver_tensor['x']}
    return tf.estimator.export.ServingInputReceiver(features, receiver_tensor)

cnn_classifier.export_savedmodel(
    export_dir_base=cnn_dir,serving_input_receiver_fn=serving_input_receiver_fn)